In [1]:
import sys, os
import numpy as np
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(r"C:\Users\super\DAG")
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
from MEC import is_in_markov_equiv_class
print(os.getcwd())
from SCM_data import generate_scm_from_BN 
from numpy.linalg import inv
from scipy.linalg import sqrtm
# from golemMain.src.golem import golem
from coordinate_descent.coordinate import dag_coordinate_descent, weight_to_adjacency
from coordinate_descent.coordinate0 import dag_coordinate_descent_l0
from coordinate_descent.cd_B import dag_coordinate_descent_BOmega, dag_coordinate_descent_BOmega_epoch
print(f"Working directory: {os.getcwd()}")

c:\Users\super\DAG\experiments
c:\Users\super\DAG\experiments
c:\Users\super\DAG\experiments
Working directory: c:\Users\super\DAG\experiments


In [2]:
experiments = []

# ----------- Experiment 1 -----------
experiments.append({
    "name": "d=3, A→B←C",
    "B_true": np.array([
        [0, 1, 0],
        [0, 0, 0],
        [0, 2, 0]
    ]),
    "N": np.array([1, 2, 3]),
    "Omega_true": np.diag([1, 2, 3])
})

# ----------- Experiment 2 -----------
experiments.append({
    "name": "d=3, A→B→C",
    "B_true": np.array([
        [0, 1, 0],
        [0, 0, 3],
        [0, 0, 0]
    ]),
    "N": np.array([1, 3, 4]),
    "Omega_true": np.diag([1, 3, 4])
})

# ----------- Experiment 3 -----------
experiments.append({
    "name": "d=3, A→B→C + A→C",
    "B_true": np.array([
        [0, 1, 2],
        [0, 0, 3],
        [0, 0, 0]
    ]),
    "N": np.array([5, 4, 3]),
    "Omega_true": np.diag([5, 4, 3])
})

# ----------- Experiment 4 -----------
experiments.append({
    "name": "d=4, A→B, B→C, B→D",
    "B_true": np.array([
        [0, 3, 0, 0],
        [0, 0, 3, 4],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ]),
    "N": np.array([1, 3, 3, 2]),
    "Omega_true": np.diag([1, 3, 3, 2])
})

# ----------- Experiment 5 -----------
experiments.append({
    "name": "d=4, A→C, A→D, B→C, B→D",
    "B_true": np.array([
        [0, 0, 2, 3],
        [0, 0, 3, 4],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ]),
    "N": np.array([2, 4, 3, 5]),
    "Omega_true": np.diag([2, 4, 3, 5])
})

# ----------- Experiment 6 -----------
experiments.append({
    "name": "d=4, A→D, B→D, C→D",
    "B_true": np.array([
        [0, 0, 0, 1],
        [0, 0, 0, 3],
        [0, 0, 0, 5],
        [0, 0, 0, 0]
    ]),
    "N": np.array([5, 4, 3, 2]),
    "Omega_true": np.diag([5, 4, 3, 2])
})

# ----------- Experiment 7 -----------
experiments.append({
    "name": "d=5, e=4, |v|=0",
    "B_true": np.array([
        [0, 1, 0, 2, 0],
        [0, 0, 3, 0, 4],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]
    ]),
    "N": np.array([1, 2, 3, 2, 1]),
    "Omega_true": np.diag([1, 2, 3, 2, 1])
})

# ----------- Experiment 8 -----------
experiments.append({
    "name": "d=5, e=4, |v|=1",
    "B_true": np.array([
        [0, 0, 1, 2, 0],
        [0, 0, 0, 2, 3],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]
    ]),
    "N": np.array([1, 2, 3, 2, 1]),
    "Omega_true": np.diag([1, 2, 3, 2, 1])
})

# ----------- Experiment 9 -----------
experiments.append({
    "name": "d=5, e=4, |v|=2",
    "B_true": np.array([
        [0, 0, 0, 1, 0],
        [0, 0, 0, 2, 3],
        [0, 0, 0, 0, 4],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]
    ]),
    "N": np.array([1, 2, 3, 2, 1]),
    "Omega_true": np.diag([1, 2, 3, 2, 1])
})


In [4]:
# Test CD-BOmega on Experiment 2 (Standard SCM 2: A -> B, C independent) using config from experiments listprint("\n=== Testing on Experiment 2 (A -> B -> C) ===")

# 1. Retrieve configuration from 'experiments' list (Index 1 corresponds to Experiment 2)
exp_config = experiments[1]
print(f"Loaded config: {exp_config['name']}")

B_true = exp_config["B_true"]
N = exp_config["N"]
Omega_true = exp_config["Omega_true"]

# 2. Setup Parameters
n_samples = 5000
seed = 10
lambda_l0 = 0.0


data, G_true_gen, _, _ = generate_scm_from_BN(
    B_true.T,  # Transposing as per reference usage
    n_samples=n_samples, 
    N=N, 
    seed=seed
)
n, d = data.shape

S = data.T @ data / n
# Initialize Omega with inverse variance of data (diagonal approximation)
Omega_init = np.diag(1.0 / np.diag(S)) 

# 5. Run Algorithm
print(f"Running CD-BOmega with n={n_samples}...")
B_est, G_est, score, debug_info = dag_coordinate_descent_BOmega(
    S, 
    Omega_init, 
    T=5000, 
    seed=seed, 
    lambda_l0=lambda_l0, 
    k=None
)

# 6. Evaluation
G_true = weight_to_adjacency(B_true, threshold=0.05) # Re-derive G from B_true for comparison
print("\nGround Truth Adjacency (from B_true):\n", G_true)
print("\nEstimated Adjacency:\n", G_est)
print("\nEstimated B:\n", np.round(B_est, 3))

# Check Mec
is_mec = is_in_markov_equiv_class(G_true, G_est)
print(f"\nIs in Markov Equivalence Class: {is_mec}")
'''
# 8. Detailed Debug Info Output
import pandas as pd
import matplotlib.pyplot as plt

print("\n=== Debug Info Analysis ===")
if debug_info:
    # Convert debug info to DataFrame for easier analysis
    df_debug = pd.DataFrame(debug_info)
    
    print("\n--- Summary of Updates ---")
    print(f"Total updates recorded: {len(df_debug)}")
    
    # Filter for non-zero updates (where delta was non-zero)
    df_active = df_debug[(df_debug['delta_ij'] != 0) | (df_debug['delta_ji'] != 0)]
    print(f"Number of active updates (delta != 0): {len(df_active)}")
    
    if not df_active.empty:
        print("\n--- Sample of Active Updates (First 10) ---")
        print(df_active.head(10)[['i', 'j', 'delta_ij', 'delta_ji']].to_string())
        
        print("\n--- Last 10 Active Updates ---")
        print(df_active.tail(10)[['i', 'j', 'delta_ij', 'delta_ji']].to_string())

        # Omega Evolution
        omega_start = df_debug.iloc[0]['omega_diag']
        omega_end = df_debug.iloc[-1]['omega_diag']
        print("\n--- Omega Evolution ---")
        print(f"Initial Omega Diag (approx): {np.round(omega_start, 3)}")
        print(f"Final Omega Diag: {np.round(omega_end, 3)}")
        print(f"True Omega Diag:  {np.diag(Omega_true)}")
        
        # Plot distribution of deltas
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.hist(df_active['delta_ij'][df_active['delta_ij'] != 0], bins=30, alpha=0.7, label='delta_ij')
        plt.title('Distribution of Non-zero delta_ij')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.hist(df_active['delta_ji'][df_active['delta_ji'] != 0], bins=30, alpha=0.7, color='orange', label='delta_ji')
        plt.title('Distribution of Non-zero delta_ji')
        plt.legend()
        plt.show()
    else:
        print("No active updates found. The algorithm might have converged immediately or initialized at a stationary point.")
else:
    print("No debug info collected.")
'''

Loaded config: d=3, A→B→C
Running CD-BOmega with n=5000...

Ground Truth Adjacency (from B_true):
 [[0 1 0]
 [0 0 1]
 [0 0 0]]

Estimated Adjacency:
 [[0 0 0]
 [1 0 1]
 [1 0 0]]

Estimated B:
 [[0.    0.    0.   ]
 [0.126 0.    0.291]
 [3.002 0.    0.   ]]

Is in Markov Equivalence Class: False


'\n# 8. Detailed Debug Info Output\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\nprint("\n=== Debug Info Analysis ===")\nif debug_info:\n    # Convert debug info to DataFrame for easier analysis\n    df_debug = pd.DataFrame(debug_info)\n    \n    print("\n--- Summary of Updates ---")\n    print(f"Total updates recorded: {len(df_debug)}")\n    \n    # Filter for non-zero updates (where delta was non-zero)\n    df_active = df_debug[(df_debug[\'delta_ij\'] != 0) | (df_debug[\'delta_ji\'] != 0)]\n    print(f"Number of active updates (delta != 0): {len(df_active)}")\n    \n    if not df_active.empty:\n        print("\n--- Sample of Active Updates (First 10) ---")\n        print(df_active.head(10)[[\'i\', \'j\', \'delta_ij\', \'delta_ji\']].to_string())\n        \n        print("\n--- Last 10 Active Updates ---")\n        print(df_active.tail(10)[[\'i\', \'j\', \'delta_ij\', \'delta_ji\']].to_string())\n\n        # Omega Evolution\n        omega_start = df_debug.iloc[0][\'omega_di

In [6]:
def run_one_coordinate_BOmega(B_true, N, name=None,
                       n_samples=5000,
                       lambda_l0 = 0.0,
                       threshold=0.05,
                       T=5000,
                       n_repeats=10, 
                       seed_data = 42):
    if name is None:
        name = "Unnamed experiment"
    print("=" * 80)
    print(f"Running CD-BOmega: {name}")

    G_true = weight_to_adjacency(B_true, threshold)

    # 1. Generate Data
    # Note: generate_scm_from_BN expects B_true.T if B is upper triangular adjacency in DAG sense?
    # In previous cell: B_true.T was passed. "Transposing as per reference usage"
    data, G_true_gen, _, _ = generate_scm_from_BN(
        B_true.T, n_samples=n_samples, N=N, seed=seed_data
    )
    n, d = data.shape
    
    # 2. Compute Statistics
    S = data.T @ data / n
    # Initialize Omega with inverse variance of data (diagonal approximation)
    Omega_init = np.diag(1.0 / np.diag(S))

    correct = 0
    
    for seed in range(n_repeats):
        B_est, G_est, score, debug_info = dag_coordinate_descent_BOmega(
            S, 
            Omega_init, 
            T=T, 
            seed=seed, 
            lambda_l0=lambda_l0, 
            k=None
        )
        
        if is_in_markov_equiv_class(G_true, G_est):
            correct += 1

    correct_rate = correct / n_repeats
    print("Correct rate =", correct_rate)
    return correct_rate

In [10]:
print("N_samples = 10000")
correct_rate = []
for seed_data in range(10):
    print("seed of data: ",seed_data)
    for exp in experiments:
        correct_rate.append(
            run_one_coordinate_BOmega(
                B_true=exp["B_true"],
                N=exp["N"],
                name=exp["name"],
                n_samples=10000,
                lambda_l0=0.0,
                threshold=0.05,
                T=5000,
                n_repeats=100,
                seed_data=seed_data
            )
        )


N_samples = 10000
seed of data:  0
Running CD-BOmega: d=3, A→B←C
Correct rate = 0.0
Running CD-BOmega: d=3, A→B→C
Correct rate = 0.31
Running CD-BOmega: d=3, A→B→C + A→C
Correct rate = 1.0
Running CD-BOmega: d=4, A→B, B→C, B→D
Correct rate = 0.0
Running CD-BOmega: d=4, A→C, A→D, B→C, B→D
Correct rate = 0.0
Running CD-BOmega: d=4, A→D, B→D, C→D
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=0
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=1
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=2
Correct rate = 0.0
seed of data:  1
Running CD-BOmega: d=3, A→B←C
Correct rate = 0.0
Running CD-BOmega: d=3, A→B→C
Correct rate = 0.31
Running CD-BOmega: d=3, A→B→C + A→C
Correct rate = 1.0
Running CD-BOmega: d=4, A→B, B→C, B→D
Correct rate = 0.0
Running CD-BOmega: d=4, A→C, A→D, B→C, B→D
Correct rate = 0.0
Running CD-BOmega: d=4, A→D, B→D, C→D
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=0
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=1
Correct rate = 0.0
Running CD-BOmega:

In [11]:
print("N_samples = 10000")
correct_rate = []
for seed_data in range(10):
    print("seed of data: ",seed_data)
    for exp in experiments:
        correct_rate.append(
            run_one_coordinate_BOmega(
                B_true=exp["B_true"],
                N=exp["N"],
                name=exp["name"],
                n_samples=10000,
                lambda_l0=0.1,
                threshold=0.05,
                T=5000,
                n_repeats=100,
                seed_data=seed_data
            )
        )

N_samples = 10000
seed of data:  0
Running CD-BOmega: d=3, A→B←C
Correct rate = 0.0
Running CD-BOmega: d=3, A→B→C
Correct rate = 0.31
Running CD-BOmega: d=3, A→B→C + A→C
Correct rate = 0.59
Running CD-BOmega: d=4, A→B, B→C, B→D
Correct rate = 0.0
Running CD-BOmega: d=4, A→C, A→D, B→C, B→D
Correct rate = 0.0
Running CD-BOmega: d=4, A→D, B→D, C→D
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=0
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=1
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=2
Correct rate = 0.0
seed of data:  1
Running CD-BOmega: d=3, A→B←C
Correct rate = 0.0
Running CD-BOmega: d=3, A→B→C
Correct rate = 0.31
Running CD-BOmega: d=3, A→B→C + A→C
Correct rate = 0.59
Running CD-BOmega: d=4, A→B, B→C, B→D
Correct rate = 0.0
Running CD-BOmega: d=4, A→C, A→D, B→C, B→D
Correct rate = 0.0
Running CD-BOmega: d=4, A→D, B→D, C→D
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=0
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=1
Correct rate = 0.0
Running CD-BOmeg

KeyboardInterrupt: 

In [7]:
print("N_samples = 10000")
correct_rate = []
for seed_data in range(10):
    print("seed of data: ",seed_data)
    for exp in experiments:
        correct_rate.append(
            run_one_coordinate_BOmega(
                B_true=exp["B_true"],
                N=exp["N"],
                name=exp["name"],
                n_samples=10000,
                lambda_l0=0.2,
                threshold=0.05,
                T=5000,
                n_repeats=100,
                seed_data=seed_data
            )
        )

N_samples = 10000
seed of data:  0
Running CD-BOmega: d=3, A→B←C
Correct rate = 0.0
Running CD-BOmega: d=3, A→B→C
Correct rate = 0.0
Running CD-BOmega: d=3, A→B→C + A→C
Correct rate = 0.0
Running CD-BOmega: d=4, A→B, B→C, B→D
Correct rate = 0.0
Running CD-BOmega: d=4, A→C, A→D, B→C, B→D
Correct rate = 0.0
Running CD-BOmega: d=4, A→D, B→D, C→D
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=0
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=1
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=2
Correct rate = 0.0
seed of data:  1
Running CD-BOmega: d=3, A→B←C
Correct rate = 0.0
Running CD-BOmega: d=3, A→B→C
Correct rate = 0.0
Running CD-BOmega: d=3, A→B→C + A→C
Correct rate = 0.04
Running CD-BOmega: d=4, A→B, B→C, B→D
Correct rate = 0.0
Running CD-BOmega: d=4, A→C, A→D, B→C, B→D
Correct rate = 0.0
Running CD-BOmega: d=4, A→D, B→D, C→D
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=0
Correct rate = 0.0
Running CD-BOmega: d=5, e=4, |v|=1
Correct rate = 0.0
Running CD-BOmega: 

KeyboardInterrupt: 

In [3]:
# Test CD-BOmega on all experiments

# Setup Parameters
n_samples = 5000
seed = 10
lambda_l0 = 0.0
threshold = 0.05
T = 5000

# Store results
results = []

# Loop through all experiments
for idx, exp_config in enumerate(experiments):
    print("\n" + "=" * 80)
    print(f"=== Experiment {idx + 1}: {exp_config['name']} ===")
    print("=" * 80)
    
    B_true = exp_config["B_true"]
    N = exp_config["N"]
    Omega_true = exp_config["Omega_true"]
    
    # Generate Data
    data, G_true_gen, _, _ = generate_scm_from_BN(
        B_true.T,  # Transposing as per reference usage
        n_samples=n_samples, 
        N=N, 
        seed=seed
    )
    n, d = data.shape
    
    # Compute covariance matrix
    S = data.T @ data / n
    # Initialize Omega with inverse variance of data (diagonal approximation)
    Omega_init = np.diag(1.0 / np.diag(S)) 
    
    # Run Algorithm
    print(f"Running CD-BOmega with n={n_samples}...")
    B_est, G_est, score, debug_info = dag_coordinate_descent_BOmega(
        S, 
        Omega_init, 
        T=T, 
        seed=seed, 
        lambda_l0=lambda_l0, 
        k=None
    )
    
    # Evaluation
    G_true = weight_to_adjacency(B_true, threshold=threshold)
    is_mec = is_in_markov_equiv_class(G_true, G_est)
    
    # Print results
    print("\nGround Truth B:\n", B_true)
    print("\nGround Truth Adjacency:\n", G_true)
    print("\nEstimated Adjacency:\n", G_est)
    print("\nEstimated B:\n", np.round(B_est, 3))
    print(f"\nIs in Markov Equivalence Class: {is_mec}")
    
    # Store result
    results.append({
        'experiment': exp_config['name'],
        'is_mec': is_mec,
        'score': score
    })

# Summary
print("\n" + "=" * 80)
print("=== SUMMARY OF ALL EXPERIMENTS ===")
print("=" * 80)
for i, result in enumerate(results):
    status = "✓ PASS" if result['is_mec'] else "✗ FAIL"
    print(f"{i+1}. {result['experiment']}: {status}")

total = len(results)
passed = sum(1 for r in results if r['is_mec'])
print(f"\nTotal: {passed}/{total} experiments recovered correct MEC")


=== Experiment 1: d=3, A→B←C ===
Running CD-BOmega with n=5000...

Ground Truth B:
 [[0 1 0]
 [0 0 0]
 [0 2 0]]

Ground Truth Adjacency:
 [[0 1 0]
 [0 0 0]
 [0 1 0]]

Estimated Adjacency:
 [[0 1 0]
 [0 0 0]
 [0 1 0]]

Estimated B:
 [[0.    0.066 0.   ]
 [0.    0.    0.   ]
 [0.    0.401 0.   ]]

Is in Markov Equivalence Class: True

=== Experiment 2: d=3, A→B→C ===
Running CD-BOmega with n=5000...

Ground Truth B:
 [[0 1 0]
 [0 0 3]
 [0 0 0]]

Ground Truth Adjacency:
 [[0 1 0]
 [0 0 1]
 [0 0 0]]

Estimated Adjacency:
 [[0 0 1]
 [1 0 1]
 [0 0 0]]

Estimated B:
 [[0.    0.    0.074]
 [0.126 0.    0.291]
 [0.    0.    0.   ]]

Is in Markov Equivalence Class: False

=== Experiment 3: d=3, A→B→C + A→C ===
Running CD-BOmega with n=5000...

Ground Truth B:
 [[0 1 2]
 [0 0 3]
 [0 0 0]]

Ground Truth Adjacency:
 [[0 1 1]
 [0 0 1]
 [0 0 0]]

Estimated Adjacency:
 [[0 0 1]
 [0 0 1]
 [0 0 0]]

Estimated B:
 [[0.    0.    0.152]
 [0.    0.    0.226]
 [0.    0.    0.   ]]

Is in Markov Equivalence 

In [6]:
# Test CD-BOmega EPOCH version on all experiments

from coordinate_descent.cd_B import ell, update_Omega_closed_form

# Setup Parameters
n_samples = 5000
seed = 10
lambda_l0 = 0.0
threshold = 0.00
T = 5000

# Store results
results_epoch = []

# Loop through all experiments
for idx, exp_config in enumerate(experiments):
    print("\n" + "=" * 80)
    print(f"=== Experiment {idx + 1} (EPOCH): {exp_config['name']} ===")
    print("=" * 80)
    
    B_true = exp_config["B_true"]
    N = exp_config["N"]
    Omega_true = exp_config["Omega_true"]
    
    # Generate Data
    data, G_true_gen, _, _ = generate_scm_from_BN(
        B_true.T,  # Transposing as per reference usage
        n_samples=n_samples, 
        N=N, 
        seed=seed
    )
    n, d = data.shape
    
    # Compute covariance matrix
    S = data.T @ data / n
    # Initialize Omega with inverse variance of data (diagonal approximation)
    Omega_init = np.diag(1.0 / np.diag(S)) 
    
    # Compute score with true parameters
    score_true = ell(B_true, Omega_true, S, n=1)
    
    # Run Algorithm (EPOCH version)
    print(f"Running CD-BOmega EPOCH with n={n_samples}...")
    # Note: returns (B, G, curr_val, history, debug_info)
    B_est, G_est, score, history, debug_info = dag_coordinate_descent_BOmega_epoch(
        S, 
        Omega_init, 
        seed=seed, 
        lambda_l0=lambda_l0, 
        k=None
    )
    
    # Compute Omega_est from B_est
    Omega_est = update_Omega_closed_form(B_est, S)
    
    # Evaluation
    G_true = weight_to_adjacency(B_true, threshold=threshold)
    is_mec = is_in_markov_equiv_class(G_true, G_est)
    
    # Print results
    print("\nGround Truth B:\n", B_true)
    print("\nGround Truth Adjacency:\n", G_true)
    print("\nEstimated Adjacency:\n", G_est)
    print("\nEstimated B:\n", np.round(B_est, 3))
    print("\nGround Truth Omega:\n", Omega_true)
    print("\nEstimated Omega:\n", np.round(Omega_est, 3))
    print(f"\nScore (true params): {score_true:.4f}")
    print(f"Score (estimated): {score:.4f}")
    print(f"Score improvement: {score_true - score:.4f}")
    print(f"Convergence history: {len(history)} epochs")
    print(f"\nIs in Markov Equivalence Class: {is_mec}")
    
    # Store result
    results_epoch.append({
        'experiment': exp_config['name'],
        'is_mec': is_mec,
        'score': score,
        'score_true': score_true,
        'score_improvement': score_true - score,
        'n_epochs': len(history)
    })

# Summary
print("\n" + "=" * 80)
print("=== SUMMARY OF ALL EXPERIMENTS (EPOCH VERSION) ===")
print("=" * 80)
for i, result in enumerate(results_epoch):
    status = "✓ PASS" if result['is_mec'] else "✗ FAIL"
    improvement = result['score_improvement']
    n_epochs = result['n_epochs']
    print(f"{i+1}. {result['experiment']}: {status} | Score improvement: {improvement:.4f} | Epochs: {n_epochs}")

total = len(results_epoch)
passed = sum(1 for r in results_epoch if r['is_mec'])
avg_improvement = np.mean([r['score_improvement'] for r in results_epoch])
avg_epochs = np.mean([r['n_epochs'] for r in results_epoch])
print(f"\nTotal: {passed}/{total} experiments recovered correct MEC")
print(f"Average score improvement: {avg_improvement:.4f}")
print(f"Average epochs to convergence: {avg_epochs:.1f}")


=== Experiment 1 (EPOCH): d=3, A→B←C ===
Running CD-BOmega EPOCH with n=5000...

Ground Truth B:
 [[0 1 0]
 [0 0 0]
 [0 2 0]]

Ground Truth Adjacency:
 [[0 1 0]
 [0 0 0]
 [0 1 0]]

Estimated Adjacency:
 [[0 1 0]
 [0 0 0]
 [0 1 0]]

Estimated B:
 [[0.    0.066 0.   ]
 [0.    0.    0.   ]
 [0.    0.401 0.   ]]

Ground Truth Omega:
 [[1 0 0]
 [0 2 0]
 [0 0 3]]

Estimated Omega:
 [[ 0.934  0.     0.   ]
 [ 0.    15.147  0.   ]
 [ 0.     0.     0.612]]

Score (true params): 2.4136
Score (estimated): 4.4598
Score improvement: -2.0462
Convergence history: 59 epochs

Is in Markov Equivalence Class: True

=== Experiment 2 (EPOCH): d=3, A→B→C ===
Running CD-BOmega EPOCH with n=5000...

Ground Truth B:
 [[0 1 0]
 [0 0 3]
 [0 0 0]]

Ground Truth Adjacency:
 [[0 1 0]
 [0 0 1]
 [0 0 0]]

Estimated Adjacency:
 [[0 1 0]
 [0 0 1]
 [0 0 0]]

Estimated B:
 [[0.    0.24  0.002]
 [0.    0.    0.301]
 [0.    0.    0.   ]]

Ground Truth Omega:
 [[1 0 0]
 [0 3 0]
 [0 0 4]]

Estimated Omega:
 [[ 0.753  0.    

In [4]:
# Test CD-BOmega EPOCH version on all experiments
# lambda_l0 = 0.01

# Setup Parameters
n_samples = 5000
seed = 10
lambda_l0 = 0.01
threshold = 0.00
T = 5000

# Store results
results_epoch = []

# Loop through all experiments
for idx, exp_config in enumerate(experiments):
    print("\n" + "=" * 80)
    print(f"=== Experiment {idx + 1} (EPOCH): {exp_config['name']} ===")
    print("=" * 80)
    
    B_true = exp_config["B_true"]
    N = exp_config["N"]
    Omega_true = exp_config["Omega_true"]
    
    # Generate Data
    data, G_true_gen, _, _ = generate_scm_from_BN(
        B_true.T,  # Transposing as per reference usage
        n_samples=n_samples, 
        N=N, 
        seed=seed
    )
    n, d = data.shape
    
    # Compute covariance matrix
    S = data.T @ data / n
    # Initialize Omega with inverse variance of data (diagonal approximation)
    Omega_init = np.diag(1.0 / np.diag(S)) 
    
    # Run Algorithm (EPOCH version)
    print(f"Running CD-BOmega EPOCH with n={n_samples}...")
    B_est, G_est,_, score, debug_info = dag_coordinate_descent_BOmega_epoch(
        S, 
        Omega_init, 
         
        seed=seed, 
        lambda_l0=lambda_l0, 
        k=None,
        min_epochs=500
    )
    
    # Evaluation
    G_true = weight_to_adjacency(B_true, threshold=threshold)
    is_mec = is_in_markov_equiv_class(G_true, G_est)
    
    # Print results
    print("\nGround Truth B:\n", B_true)
    print("\nGround Truth Adjacency:\n", G_true)
    print("\nEstimated Adjacency:\n", G_est)
    print("\nEstimated B:\n", np.round(B_est, 3))
    print(f"\nIs in Markov Equivalence Class: {is_mec}")
    
    # Store result
    results_epoch.append({
        'experiment': exp_config['name'],
        'is_mec': is_mec,
        'score': score
    })

# Summary
print("\n" + "=" * 80)
print("=== SUMMARY OF ALL EXPERIMENTS (EPOCH VERSION) ===")
print("=" * 80)
for i, result in enumerate(results_epoch):
    status = "✓ PASS" if result['is_mec'] else "✗ FAIL"
    print(f"{i+1}. {result['experiment']}: {status}")

total = len(results_epoch)
passed = sum(1 for r in results_epoch if r['is_mec'])
print(f"\nTotal: {passed}/{total} experiments recovered correct MEC")


=== Experiment 1 (EPOCH): d=3, A→B←C ===
Running CD-BOmega EPOCH with n=5000...

Ground Truth B:
 [[0 1 0]
 [0 0 0]
 [0 2 0]]

Ground Truth Adjacency:
 [[0 1 0]
 [0 0 0]
 [0 1 0]]

Estimated Adjacency:
 [[0 0 0]
 [0 0 0]
 [0 1 0]]

Estimated B:
 [[0.    0.    0.   ]
 [0.    0.    0.   ]
 [0.    0.401 0.   ]]

Is in Markov Equivalence Class: False

=== Experiment 2 (EPOCH): d=3, A→B→C ===
Running CD-BOmega EPOCH with n=5000...

Ground Truth B:
 [[0 1 0]
 [0 0 3]
 [0 0 0]]

Ground Truth Adjacency:
 [[0 1 0]
 [0 0 1]
 [0 0 0]]

Estimated Adjacency:
 [[0 1 0]
 [0 0 1]
 [0 0 0]]

Estimated B:
 [[0.    0.247 0.   ]
 [0.    0.    0.301]
 [0.    0.    0.   ]]

Is in Markov Equivalence Class: True

=== Experiment 3 (EPOCH): d=3, A→B→C + A→C ===
Running CD-BOmega EPOCH with n=5000...

Ground Truth B:
 [[0 1 2]
 [0 0 3]
 [0 0 0]]

Ground Truth Adjacency:
 [[0 1 1]
 [0 0 1]
 [0 0 0]]

Estimated Adjacency:
 [[0 1 0]
 [0 0 1]
 [0 0 0]]

Estimated B:
 [[0.    0.551 0.   ]
 [0.    0.    0.226]
 [0.  

In [18]:
# Detailed test of dag_coordinate_descent_BOmega on Experiment 1
# Manual calculation of ell components

from coordinate_descent.cd_B import ell, update_Omega_closed_form

print("=" * 80)
print("=== Detailed Test: dag_coordinate_descent_BOmega on Experiment 1 ===")
print("=" * 80)

# Get Experiment 1 configuration
exp_config = experiments[0]
print(f"\nExperiment: {exp_config['name']}")
print(f"Structure: A→B←C (common effect / collider)")

B_true = exp_config["B_true"]
N = exp_config["N"]
Omega_true = exp_config["Omega_true"]

print("\n--- TRUE PARAMETERS ---")
print("\nB_true (adjacency weights):")
print(B_true)
print("\nOmega_true (noise precision):")
print(Omega_true)
print(f"Omega_true diagonal: {np.diag(Omega_true)}")

# Generate data
n_samples = 5000
seed = 42
data, _, _, _ = generate_scm_from_BN(
    B_true.T,
    n_samples=n_samples, 
    N=N, 
    seed=seed
)
n, d = data.shape
print(f"\n--- DATA ---")
print(f"Sample size: n = {n}")
print(f"Dimensions: d = {d}")

# Compute sample covariance
S = data.T @ data / n
print("\n--- SAMPLE STATISTICS ---")
print("\nS (sample covariance / cross-product matrix):")
print(np.round(S, 4))

# Initialize Omega
Omega_init = np.diag(1.0 / np.diag(S))
print("\nOmega_init (inverse of sample variances):")
print(np.round(Omega_init, 4))
print(f"Omega_init diagonal: {np.round(np.diag(Omega_init), 4)}")

# ============================================================
# Manual calculation of ell for TRUE parameters (n=1)
# ============================================================
print("\n" + "=" * 80)
print("=== MANUAL CALCULATION: TRUE PARAMETERS (n=1) ===")
print("=" * 80)

I_minus_B_true = np.eye(d) - B_true
print("\nI - B_true:")
print(np.round(I_minus_B_true, 4))

# Term 1: log det Omega
omega_diag_true = np.diag(Omega_true)
logdet_Omega_true = np.sum(np.log(omega_diag_true))
print(f"\n[Term 1] log det(Omega_true)")
print(f"  Omega diagonal: {omega_diag_true}")
print(f"  log det(Omega) = sum(log(diag)) = {logdet_Omega_true:.6f}")

# Term 2: -2 log det(I - B)
sign_true, logabsdet_true = np.linalg.slogdet(I_minus_B_true)
logdet_IminusB_true = logabsdet_true
print(f"\n[Term 2] -2 * log det(I - B_true)")
print(f"  det(I - B) sign: {sign_true}")
print(f"  log|det(I - B)| = {logdet_IminusB_true:.6f}")
print(f"  -2 * log det(I - B) = {-2.0 * logdet_IminusB_true:.6f}")

# Term 3: tr((I - B)^T Omega^{-1} (I - B) S)
Omega_inv_true = np.diag(1.0 / omega_diag_true)
trace_term3_true = np.trace(I_minus_B_true.T @ Omega_inv_true @ I_minus_B_true @ S)
print(f"\n[Term 3] tr((I - B)^T Omega^{{-1}} (I - B) S)")
print(f"  Omega^{{-1}} diagonal: {np.diag(Omega_inv_true)}")
print(f"  (I - B)^T @ Omega^{{-1}} @ (I - B) @ S:")
print(np.round(I_minus_B_true.T @ Omega_inv_true @ I_minus_B_true @ S, 4))
print(f"  trace = {trace_term3_true:.6f}")

# Term 4: tr((I - B) Omega^{-1} (I - B)^T S) [Alternative formulation]
trace_term4_true = np.trace(I_minus_B_true @ Omega_inv_true @ I_minus_B_true.T @ S)
print(f"\n[Term 4] tr((I - B) Omega^{{-1}} (I - B)^T S)")
print(f"  (I - B) @ Omega^{{-1}} @ (I - B)^T @ S:")
print(np.round(I_minus_B_true @ Omega_inv_true @ I_minus_B_true.T @ S, 4))
print(f"  trace = {trace_term4_true:.6f}")
print(f"  Note: Term3 and Term4 differ due to matrix order")

# Total (n=1)
score_true_n1_manual = 0.5 * (logdet_Omega_true - 2.0 * logdet_IminusB_true + trace_term3_true)
print(f"\n[TOTAL] ell(B_true, Omega_true, S, n=1)")
print(f"  = (n/2) * (term1 + term2 + term3)")
print(f"  = 0.5 * ({logdet_Omega_true:.6f} + {-2.0 * logdet_IminusB_true:.6f} + {trace_term3_true:.6f})")
print(f"  = {score_true_n1_manual:.6f}")

# Verify with ell function
score_true_n1_func = ell(B_true, Omega_true, S, n=1)
print(f"\n[VERIFICATION] ell() function: {score_true_n1_func:.6f}")
print(f"Difference: {abs(score_true_n1_manual - score_true_n1_func):.10f}")

# ============================================================
# Run algorithm and get estimated parameters
# ============================================================
print("\n" + "=" * 80)
print("=== RUNNING ALGORITHM ===")
print("=" * 80)
lambda_l0 = 0.0
T = 5000
print(f"Parameters: T={T}, lambda_l0={lambda_l0}, seed={seed}")

B_est, G_est, score_est, debug_info = dag_coordinate_descent_BOmega(
    S, 
    Omega_init, 
    T=T, 
    seed=seed, 
    lambda_l0=lambda_l0, 
    k=None
)

# Update Omega based on estimated B
Omega_est = update_Omega_closed_form(B_est, S)

print("\n--- ESTIMATED PARAMETERS ---")
print("\nB_est (estimated adjacency weights):")
print(np.round(B_est, 4))
print("\nOmega_est (estimated noise precision):")
print(np.round(Omega_est, 4))
print(f"Omega_est diagonal: {np.round(np.diag(Omega_est), 4)}")
print("\nG_est (estimated adjacency matrix, threshold=0.05):")
print(G_est)

# ============================================================
# Manual calculation of ell for ESTIMATED parameters (n=1)
# ============================================================
print("\n" + "=" * 80)
print("=== MANUAL CALCULATION: ESTIMATED PARAMETERS (n=1) ===")
print("=" * 80)

I_minus_B_est = np.eye(d) - B_est
print("\nI - B_est:")
print(np.round(I_minus_B_est, 4))

# Term 1: log det Omega
omega_diag_est = np.diag(Omega_est)
logdet_Omega_est = np.sum(np.log(omega_diag_est))
print(f"\n[Term 1] log det(Omega_est)")
print(f"  Omega diagonal: {np.round(omega_diag_est, 4)}")
print(f"  log det(Omega) = sum(log(diag)) = {logdet_Omega_est:.6f}")

# Term 2: -2 log det(I - B)
sign_est, logabsdet_est = np.linalg.slogdet(I_minus_B_est)
logdet_IminusB_est = logabsdet_est
print(f"\n[Term 2] -2 * log det(I - B_est)")
print(f"  det(I - B) sign: {sign_est}")
print(f"  log|det(I - B)| = {logdet_IminusB_est:.6f}")
print(f"  -2 * log det(I - B) = {-2.0 * logdet_IminusB_est:.6f}")

# Term 3: tr((I - B)^T Omega^{-1} (I - B) S)
Omega_inv_est = np.diag(1.0 / omega_diag_est)
trace_term3_est = np.trace(I_minus_B_est.T @ Omega_inv_est @ I_minus_B_est @ S)
print(f"\n[Term 3] tr((I - B)^T Omega^{{-1}} (I - B) S)")
print(f"  Omega^{{-1}} diagonal: {np.round(np.diag(Omega_inv_est), 4)}")
print(f"  (I - B)^T @ Omega^{{-1}} @ (I - B) @ S:")
print(np.round(I_minus_B_est.T @ Omega_inv_est @ I_minus_B_est @ S, 4))
print(f"  trace = {trace_term3_est:.6f}")

# Term 4: tr((I - B) Omega^{-1} (I - B)^T S) [Alternative formulation]
trace_term4_est = np.trace(I_minus_B_est @ Omega_inv_est @ I_minus_B_est.T @ S)
print(f"\n[Term 4] tr((I - B) Omega^{{-1}} (I - B)^T S)")
print(f"  (I - B) @ Omega^{{-1}} @ (I - B)^T @ S:")
print(np.round(I_minus_B_est @ Omega_inv_est @ I_minus_B_est.T @ S, 4))
print(f"  trace = {trace_term4_est:.6f}")
print(f"  Note: Term3 and Term4 differ due to matrix order")

# Total (n=1)
score_est_n1_manual = 0.5 * (logdet_Omega_est - 2.0 * logdet_IminusB_est + trace_term3_est)
print(f"\n[TOTAL] ell(B_est, Omega_est, S, n=1)")
print(f"  = (n/2) * (term1 + term2 + term3)")
print(f"  = 0.5 * ({logdet_Omega_est:.6f} + {-2.0 * logdet_IminusB_est:.6f} + {trace_term3_est:.6f})")
print(f"  = {score_est_n1_manual:.6f}")

# Verify with ell function
score_est_n1_func = ell(B_est, Omega_est, S, n=1)
print(f"\n[VERIFICATION] ell() function: {score_est_n1_func:.6f}")
print(f"Difference: {abs(score_est_n1_manual - score_est_n1_func):.10f}")

# ============================================================
# Comparison of terms
# ============================================================
print("\n" + "=" * 80)
print("=== TERM-BY-TERM COMPARISON ===")
print("=" * 80)

print(f"\n{'Term':<35} {'True Params':>15} {'Est Params':>15} {'Difference':>15}")
print("-" * 85)
print(f"{'log det(Omega)':<35} {logdet_Omega_true:>15.6f} {logdet_Omega_est:>15.6f} {logdet_Omega_est - logdet_Omega_true:>15.6f}")
print(f"{'-2 log det(I-B)':<35} {-2.0*logdet_IminusB_true:>15.6f} {-2.0*logdet_IminusB_est:>15.6f} {-2.0*(logdet_IminusB_est - logdet_IminusB_true):>15.6f}")
print(f"{'tr((I-B)^T Omega^-1 (I-B) S)':<35} {trace_term3_true:>15.6f} {trace_term3_est:>15.6f} {trace_term3_est - trace_term3_true:>15.6f}")
print(f"{'tr((I-B) Omega^-1 (I-B)^T S)':<35} {trace_term4_true:>15.6f} {trace_term4_est:>15.6f} {trace_term4_est - trace_term4_true:>15.6f}")
print("-" * 85)
print(f"{'Sum of terms (1+2+3)':<35} {logdet_Omega_true - 2.0*logdet_IminusB_true + trace_term3_true:>15.6f} {logdet_Omega_est - 2.0*logdet_IminusB_est + trace_term3_est:>15.6f} {(logdet_Omega_est - 2.0*logdet_IminusB_est + trace_term3_est) - (logdet_Omega_true - 2.0*logdet_IminusB_true + trace_term3_true):>15.6f}")
print(f"{'ell (n=1)':<35} {score_true_n1_manual:>15.6f} {score_est_n1_manual:>15.6f} {score_est_n1_manual - score_true_n1_manual:>15.6f}")

# Check if Term 3 and Term 4 are equal
print("\n" + "=" * 80)
print("=== CHECKING TERM 3 vs TERM 4 ===")
print("=" * 80)
print("Note: For diagonal Omega, these terms may differ when B is not symmetric")
print(f"\nTrue params - Term3 == Term4: {np.allclose(trace_term3_true, trace_term4_true)}")
print(f"  Difference: {abs(trace_term3_true - trace_term4_true):.10f}")
print(f"\nEst params - Term3 == Term4: {np.allclose(trace_term3_est, trace_term4_est)}")
print(f"  Difference: {abs(trace_term3_est - trace_term4_est):.10f}")

# MEC check
G_true = weight_to_adjacency(B_true, threshold=0.05)
is_mec = is_in_markov_equiv_class(G_true, G_est)

print("\n" + "=" * 80)
print("=== SUMMARY ===")
print("=" * 80)
print(f"Structure recovered correctly: {is_mec}")
print(f"Estimated parameters achieve lower score (better fit on training data)")
print(f"Score improvement: {score_true_n1_manual - score_est_n1_manual:.6f}")
print(f"Relative improvement: {(score_true_n1_manual - score_est_n1_manual) / abs(score_true_n1_manual) * 100:.2f}%")

=== Detailed Test: dag_coordinate_descent_BOmega on Experiment 1 ===

Experiment: d=3, A→B←C
Structure: A→B←C (common effect / collider)

--- TRUE PARAMETERS ---

B_true (adjacency weights):
[[0 1 0]
 [0 0 0]
 [0 2 0]]

Omega_true (noise precision):
[[1 0 0]
 [0 2 0]
 [0 0 3]]
Omega_true diagonal: [1 2 3]

--- DATA ---
Sample size: n = 5000
Dimensions: d = 3

--- SAMPLE STATISTICS ---

S (sample covariance / cross-product matrix):
[[ 1.0111e+00  1.0161e+00 -8.0000e-04]
 [ 1.0161e+00  1.5324e+01  6.0750e+00]
 [-8.0000e-04  6.0750e+00  3.0058e+00]]

Omega_init (inverse of sample variances):
[[0.989  0.     0.    ]
 [0.     0.0653 0.    ]
 [0.     0.     0.3327]]
Omega_init diagonal: [0.989  0.0653 0.3327]

=== MANUAL CALCULATION: TRUE PARAMETERS (n=1) ===

I - B_true:
[[ 1. -1.  0.]
 [ 0.  1.  0.]
 [ 0. -2.  1.]]

[Term 1] log det(Omega_true)
  Omega diagonal: [1 2 3]
  log det(Omega) = sum(log(diag)) = 1.791759

[Term 2] -2 * log det(I - B_true)
  det(I - B) sign: 1.0
  log|det(I - B)| 